# 네이버 검색 + 블로그 본문 스크랩

## 정보 입력 / 전역 설정

### 환경 변수

In [13]:
# 실행 순번 - 경로에 사용
RUN_INDEX = 1                 # !!중요!! - 실행=파일명 순번 => 수동 업데이트

# --- 검색 일자 ---
start_date = '20250101'
end_date = ''                 # 입력 안 하면 오늘 날짜로 세팅

# --- 스크롤 횟수 ---
scroll_times = 10              # 스크롤 횟수

# --- 다운로드 옵션 ---
IS_SAVE_CSV = True            # CSV(엑셀용) 저장 여부
IS_SAVE_IMAGES = False         # 이미지 저장 여부

# --- 파일 다운로드 경로 (ROOT) --
ROOT_DOWNLOAD_DIR = 'craw-download-file'

### 검색 키워드 입력

In [2]:
# 단일 키워드
keyword = '네일추천'            # 검색 키워드
search_keyword = keyword      # 검색창에 넣을 실제 값

# 다중 키워드 (여러번 검색 수행) - 비어 있다면, 단일 키워드로 수행
search_keywords = [
    # 1) 기본/일반 네일 (베이스 분포용)
    "네일 추천",
    "네일 디자인",
    "셀프네일 디자인",
    "데일리 네일",
    "직장인 네일",

    # # 2) 쉐입 중심
    # "라운드 네일 디자인",
    # "오벌 네일 디자인",
    # "스퀘어 네일 디자인",
    # "스퀘어오벌 네일",
    # "아몬드 네일 디자인",

    # # 3) 길이·손 특징 중심
    # "짧은 손톱 네일",
    # "짧은손톱 셀프네일",
    # "긴 손톱 네일",
    # "손가락 길어보이는 네일",
    # "손이 예뻐보이는 네일",
]

## 함수 정의

### 모듈 import

In [3]:
import os
import re
import requests
from datetime import datetime
from urllib.parse import urljoin

import pandas as pd
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import mimetypes

### 파일 경로

In [4]:
## 파일 경로 : 일자 + 실행 순번 + 검색어 or multi

today_date = datetime.now().strftime("%Y%m%d")

def sanitize_keyword(kw: str) -> str:
    """파일/폴더명으로 쓸 수 있게 키워드 정제 """
    if not isinstance(kw, str):
        kw = str(kw)
    kw = kw.strip().strip('"').strip("'")
    # 파일명에 쓸 수 없는 문자 제거
    kw = re.sub(r'[\\/:*?"<>|]', '_', kw)
    # 너무 길면 잘라내기
    if len(kw) == 0:
        kw = "keyword"
    return kw

### 단일/다중에 따라 파일명 기준 키워드 결정
if search_keywords:          # 다중 키워드 리스트가 비어있지 않다면
    base_keyword_for_path = "multi"
else:
    base_keyword_for_path = search_keyword

SAFE_KEYWORD = sanitize_keyword(base_keyword_for_path)

SUB_NAME = f"{today_date}_{RUN_INDEX:02d}_{SAFE_KEYWORD}"

CSV_PATH = os.path.join(ROOT_DOWNLOAD_DIR, f"{SUB_NAME}.csv")
IMAGE_DIR = os.path.join(ROOT_DOWNLOAD_DIR, SUB_NAME)

### 날짜 정규화

In [5]:
import re
from datetime import datetime, timedelta, timezone

KST = timezone(timedelta(hours=9))

_abs = re.compile(r"^\s*(\d{4})\.(\d{1,2})\.(\d{1,2})\.?\s*$")
_m  = re.compile(r"(\d+)\s*분\s*전")
_h  = re.compile(r"(\d+)\s*시간\s*전")
_d  = re.compile(r"(\d+)\s*일\s*전")
_w  = re.compile(r"(\d+)\s*주\s*전")
_not_date_tail = re.compile(r"(?:단|TOP),\s*$")        # 예: "A34면 1단," , "A30면 TOP,"

def _ymd(dt: datetime) -> str:
    # Python/Excel 공통 인식 포맷: YYYY-MM-DD
    return f"{dt.year:04d}-{dt.month:02d}-{dt.day:02d}"

def normalize_news_date(texts, now: datetime | None = None) -> str | None:
    # 변환 실패 대비: 원본 첫 값 보관
    original_first = texts if isinstance(texts, str) else (texts[0] if texts else None)

    if isinstance(texts, str):
        texts = [texts]
    now = now or datetime.now(KST)

    for t in texts:
        if not t:
            continue
        s = t.strip()

        # 지면/면·단, TOP 꼬리면 스킵
        if _not_date_tail.search(s):
            continue

        # 절대형
        m = _abs.match(s)
        if m:
            y, mo, d = map(int, m.groups())
            return f"{y:04d}-{mo:02d}-{d:02d}"

        # 상대형
        if (mm := _h.search(s)): # 분
            return _ymd(now - timedelta(minutes=int(mm.group(1))))
        if (mh := _h.search(s)): # 시간
            return _ymd(now - timedelta(hours=int(mh.group(1))))
        if (md := _d.search(s)): # 일
            return _ymd(now - timedelta(days=int(md.group(1))))
        if (mw := _w.search(s)): # 주
            return _ymd(now - timedelta(weeks=int(mw.group(1))))

    # 날짜형 변환이 안되면 첫 번째 값 반환
    return original_first

### 검색 결과 카드 한 개에서 정보 추출

In [6]:
def get_content(review):
    condic = {}

    title_css  = (
        "span.sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1"
        ".sds-comps-text-type-headline1.sds-comps-text-weight-sm"
    )
    text_css   = (
        "span.sds-comps-text-type-body1"
    )
    viewer_css = (
        "div.sds-comps-horizontal-layout.sds-comps-inline-layout"
        ".sds-comps-profile-info-title"
    )
    date_css   = (
        "span.sds-comps-text.sds-comps-text-type-body2"
        ".sds-comps-text-weight-sm.sds-comps-profile-info-subtext"
    )
    link_css   = 'a[href*="blog.naver.com"]'

    condic['writer'] = review.find_element(By.CSS_SELECTOR, viewer_css) \
                             .get_attribute('innerText').strip()
    condic['title'] = review.find_element(By.CSS_SELECTOR, title_css) \
                             .get_attribute('innerText').strip()
    condic['text'] = review.find_element(By.CSS_SELECTOR, text_css) \
                            .get_attribute('innerText').strip()
    condic['date'] = normalize_news_date(
        review.find_element(By.CSS_SELECTOR, date_css)
              .get_attribute('innerText').strip()
    )
    try:
        condic['link'] = review.find_element(By.CSS_SELECTOR, link_css) \
                                .get_attribute('href')
    except Exception:
        condic['link'] = ''
        print(f"condic['link'] -> 추출오류 : {condic['title']}")

    return condic

### 블로그 본문 크롤링 + 이미지 저장

In [7]:
def crawl_blog_content(url: str, search_idx: int,
                       image_dir: str,
                       is_download_images: bool = True):

    COMMON_HEADERS = {"User-Agent": "Mozilla/5.0"}

    resp = requests.get(url, headers=COMMON_HEADERS)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # iframe 찾기
    iframe = soup.select_one("iframe#mainFrame")
    if iframe is None:
        for f in soup.find_all("iframe"):
            src = f.get("src") or ""
            if "PostView" in src:
                iframe = f
                break

    if iframe is None:
        print(f"[WARN] iframe 없음 → {url}")
        return {"title": "", "full_text": "", "image_count": 0}

    inner_src = iframe.get("src")
    inner_url = urljoin(resp.url, inner_src)

    # iframe 안쪽 요청
    resp2 = requests.get(inner_url, headers=COMMON_HEADERS)
    resp2.raise_for_status()
    soup2 = BeautifulSoup(resp2.text, "html.parser")

    # 제목
    title_tag = soup2.find("title")
    title = title_tag.get_text(strip=True) if title_tag else ""

    # 본문 텍스트
    paragraphs = [p.get_text(" ", strip=True) for p in soup2.find_all("p")]
    full_text = "\n".join(paragraphs)

    if not full_text.strip():
        main_div = soup2.select_one("div.se-main-container")
        if main_div:
            paragraphs = [
                p.get_text(" ", strip=True)
                for p in main_div.find_all(["p", "span"])
            ]
            full_text = "\n".join(paragraphs)

    # 이미지 다운로드
    image_count = 0
    if is_download_images:
        content = (
            soup2.select_one("div.se-main-container")
            or soup2.select_one("#postViewArea")
            or soup2
        )
        img_tags = content.find_all("img")

        os.makedirs(image_dir, exist_ok=True)

        allowed_hosts = ("postfiles.pstatic.net", "blogfiles.pstatic.net")

        for img_idx, img in enumerate(img_tags, start=1):
            src = img.get("data-origin") or img.get("data-src") or img.get("src")
            if not src:
                continue

            if not any(h in src for h in allowed_hosts):
                continue

            if src.startswith("//"):
                src = "https:" + src
            elif src.startswith("/"):
                src = urljoin(inner_url, src)
            if src.startswith("data:"):
                continue

            hi_src = src
            if "type=w80_blur" in src:
                hi_src = src.replace("type=w80_blur", "type=w966")

            img_headers = {
                "User-Agent": "Mozilla/5.0",
                "Referer": inner_url,
            }

            img_resp = None
            for candidate in [hi_src, src]:
                try:
                    r = requests.get(candidate, headers=img_headers, timeout=10)
                    r.raise_for_status()
                    img_resp = r
                    break
                except Exception:
                    img_resp = None

            if img_resp is None:
                continue

            content_type = img_resp.headers.get("Content-Type", "")
            if "image" not in content_type:
                continue

            if not is_download_images:
                image_count += 1
                continue

            ext = mimetypes.guess_extension(content_type.split(";")[0]) or ".jpg"

            # IMG_{검색결과순번 4자리}_{본문내 이미지순번 4자리}
            filename = os.path.join(
                image_dir,
                f"IMG_{search_idx:04d}_{img_idx:04d}{ext}"
            )

            with open(filename, "wb") as f:
                f.write(img_resp.content)

            image_count += 1

    return {
        "title": title,
        "full_text": full_text,
        "image_count": image_count,
    }


def enrich_with_blog_contents(df_search: pd.DataFrame) -> pd.DataFrame:
    """
    1번 검색 df 기준:
    - title 덮어쓰기 (블로그 실제 제목)
    - full_text 열 추가
    - image_count 열 추가
    """

    df = df_search.copy()
    df["full_text"] = ""
    df["image_count"] = 0

    for idx, row in df.iterrows():
        url = row.get("link", "")
        if not isinstance(url, str) or not url.strip():
            continue

        search_idx = idx + 1
        print(f"[{search_idx}/{len(df)}] {url}")

        info = crawl_blog_content(
            url=url,
            search_idx=search_idx,
            image_dir=IMAGE_DIR,
            is_download_images=IS_SAVE_IMAGES
        )

        df.at[idx, "title"] = info["title"]
        df.at[idx, "full_text"] = info["full_text"]
        df.at[idx, "image_count"] = info["image_count"]

    return df

## 실행부

### 검색 페이지 로드 / 스크롤 / 본문 스크랩

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from urllib.parse import quote_plus

from time import sleep
import random

def encode_keyword(search_keyword: str) -> str:
    return quote_plus(search_keyword)

driver = webdriver.Chrome()

# 다중 키워드가 있으면 그걸 사용, 없으면 단일 search_keyword 사용
if search_keywords:
    keywords_to_run = search_keywords
else:
    keywords_to_run = [search_keyword]

all_results = []

for q in keywords_to_run:
    print(f"\n=== 검색 시작: {q} ===")
    enc_q = encode_keyword(q)

    url = (
        "https://search.naver.com/search.naver"
        "?ssc=tab.blog.all"
        "&sm=tab_opt"
        "&where=blog"
        f"&query={enc_q}"
        f"&nso=so%3Ar%2Cp%3Afrom{start_date}to{end_date}"
    )

    driver.get(url)

    # 스크롤
    xp = '/html'
    review_box = driver.find_element(By.XPATH, xp)

    for _ in range(scroll_times):
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', review_box)
        sleep(random.randint(2, 3))

    # 박스 선택
    forms = driver.find_elements(
        By.CSS_SELECTOR,
        'div.sds-comps-vertical-layout.sds-comps-full-layout.YNThgbNOBFUb3Wc2Cq47'
    )
    print('박스 수 :', len(forms))

    # 카드별 정보 수집
    success = 0
    # fail_stats = {}
    for i, form in enumerate(forms, start=1):
        try:
            dic = get_content(form)
            dic["search_keyword"] = q   # 어떤 검색어에서 나온 결과인지 기록
            all_results.append(dic)
            success += 1
        except Exception as e:
            name = type(e).__name__
            # fail_stats[name] = fail_stats.get(name, 0) + 1
            # print(f"[{q}] 카드 {i}/{len(forms)} skip → {name}: {e}")

    # print(f"[{q}] 성공: {success} / {len(forms)}")
    # print(f"[{q}] 실패 통계: {fail_stats}")

# 모든 결과를 하나의 DataFrame으로
df_search = pd.DataFrame(all_results)
print("총 수집 건수:", len(df_search))


df_search['idx'] = range(len(df_search))

cols = (['idx', 'search_keyword']
        + [c for c in df_search.columns if c not in ['idx', 'search_keyword']])
df_search = df_search[cols]

df_search.head()


=== 검색 시작: 네일 추천 ===
박스 수 : 126
condic['link'] -> 추출오류 : 네일아트하러 서울에서 방문하는 손더네일

=== 검색 시작: 네일 디자인 ===
박스 수 : 94
condic['link'] -> 추출오류 : 네일아트 자격증 필기 실기에 대해서

=== 검색 시작: 셀프네일 디자인 ===
박스 수 : 110

=== 검색 시작: 데일리 네일 ===
박스 수 : 128

=== 검색 시작: 직장인 네일 ===
박스 수 : 137
총 수집 건수: 595


,idx,search_keyword,writer,title,text,date,link
0,0,네일 추천,깜썰,베트남 다낭 마사지 추천 핑크스파 네일아트,"베트남 다낭 마사지 추천 핑크스파 네일아트 하잉, 여행블로거 깜썰입니다 :) 남편 ...",2025-10-10,https://blog.naver.com/jeongwon84/224036100473
1,1,네일 추천,쥔이로그 : 뷰티&패션&팝업,"자석젤 다이소 네일팁 웨딩네일 추천, 간단하게 포인트 주기","""자석젤 다이소 네일팁 웨딩네일 추천, 간단하게 포인트 주기"" 안녕하세요! 쥔이입니...",2025-08-17,https://blog.naver.com/486xhxh
2,2,네일 추천,★ 빛나는 나의 길 ★,경성대 부경대네일 랑만뷰티살롱 쿠로미네일 추천,[ 경성대 부경대네일 랑만뷰티살롱 ] 쿠로미네일 추천 쿠로미 좋아하는 사람들 다 모...,2025-11-06,https://blog.naver.com/wonnabe
3,3,네일 추천,뉴라와 구름이의 먹방 라이프⭐️,"범계 속눈썹 연장 잘하는 벨라네일 추천 (주차, 애견동반가능)",안녕하세요 뉴릉이입니다~ 저는 이번에 새로운 연장 잘 하는 곳을 찾다가 범계 속눈썹...,2025-11-17,https://blog.naver.com/our0511
4,4,네일 추천,[생각중],강남 선릉역네일 화려한네일 전문샵 도쿄네일 추천,규모가 크고 인테리어도 멋지면서 편안하게 네일을 받을 수 있는 강남 선릉역네일 화려...,2025-10-30,https://blog.naver.com/molly32


### 블로그 본문 + 이미지 수집

In [11]:
### 블로그 본문 + 이미지 수집
if IS_SAVE_IMAGES or True:   # 이미지 저장 안하더라도 본문 수집은 수행
    df_enriched = enrich_with_blog_contents(df_search)
else:
    df_enriched = df_search.copy()

[1/595] https://blog.naver.com/jeongwon84/224036100473
[2/595] https://blog.naver.com/486xhxh
[3/595] https://blog.naver.com/wonnabe
[4/595] https://blog.naver.com/our0511
[5/595] https://blog.naver.com/molly32
[6/595] https://blog.naver.com/heoyumi94
[7/595] https://blog.naver.com/darkyoon85/224075245164
[8/595] https://blog.naver.com/hw03088
[9/595] https://blog.naver.com/tjdbs0323/224077123725
[10/595] https://blog.naver.com/aprilsong19/224045784654
[12/595] https://blog.naver.com/sunkjydbgn/224079739219
[13/595] https://blog.naver.com/wh110595
[14/595] https://blog.naver.com/idb0704
[15/595] https://blog.naver.com/sbnita_bibi
[16/595] https://blog.naver.com/abalico
[17/595] https://blog.naver.com/hwang_c11
[18/595] https://blog.naver.com/k_fla
[19/595] https://blog.naver.com/diecruel/224030153990
[20/595] https://blog.naver.com/ggggggenie/224063015455
[21/595] https://blog.naver.com/banglang93
[22/595] https://blog.naver.com/sscd0808/224068978485
[23/595] https://blog.naver.com/gmj

### CSV 저장

In [12]:
### CSV 저장
if IS_SAVE_CSV:
    os.makedirs(ROOT_DOWNLOAD_DIR, exist_ok=True)
    df_enriched.to_csv(CSV_PATH, index=False, encoding="utf-8-sig")
    print("CSV 저장 완료:", CSV_PATH)

CSV 저장 완료: craw-download-file\20251120_03_multi.csv


In [ ]:
df_enriched